# **Przygotowywanie środowiska**

In [ ]:
import matplotlib.image as image
import matplotlib.pyplot as plt
import numpy as np
import zipfile as zp
import os
import pandas as pd
import torch
from tqdm.notebook import tqdm

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Etykietowanie**

In [ ]:
Faces_Dir = '/content/drive/MyDrive/ASIwP4_0_age_recognition/archive/UTKFace'

In [ ]:
img_paths =[] #for multiple pictures
age_labels = []

for filename in tqdm(os.listdir(Faces_Dir)):
  img_path = os.path.join(Faces_Dir, filename)
  tmp = filename.split('_')
  age = int(tmp[0])

  age_labels.append(age)
  img_paths.append(img_path)

In [ ]:
data_frame = pd.DataFrame()
data_frame['image'], data_frame['age']= img_paths, age_labels

In [ ]:
data_frame.head

# **Przetwarzanie na zestaw cech**

In [ ]:
def images_featuring(images):
  features = []
  for num in tqdm(range(images.size)):
    img = torchvision.io.read_image(images[num],
                                    torchvision.io.ImageReadMode.GRAY)
    img = img.reshape((200, 200), 1)
    img = img.numpy()
    features.append(img)
  features = np.array(features)
  features = features.reshape(len(features), 200, 200, 1)

  return features

In [ ]:
X = images_featuring(data_frame['image'])

In [ ]:
# Normalizacja pix
X = X/255.0

In [ ]:
# Przeniesienie tensora
X = torch.Tensor(X).to(device).view(-1,200,200)

In [ ]:
# Wykorzystanie Cuda
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# Wczytanie z drive
X = np.load("/content/drive/MyDrive/Projekt_Zespołowy/X_np.npy")

In [ ]:
# Przekształcenie np.array do tensora
X = torch.Tensor(X)
X[0]

In [ ]:
# Dane z DataFrame do tablicy
y_age = np.array(data_frame['age'])

## **Funkcja do dokładności**

In [ ]:
def get_accuracy(model, X_test, y_test):
  correct = 0
  total = 0
  with torch.no_grad():
      for i in range(len(X_test)):
          real_class = y_test[i]
          net_out = model(X_test[i].to(device).view(-1, 1, 200, 200))[0]  # returns a list,
          predicted_class = torch.argmax(net_out)

          if predicted_class == real_class:
              correct += 1
          total += 1
  return round(correct/total, 3)

# **Estymacja wieku**

In [ ]:
age_dictionary = {0:'(0-5>', 1:'(5-10>', 2:'(10-15>', 3:'(15-20>', 4:'(20-30>', 5:'(30-40>', 6:'(40-50', 7:'(50-70>', 8:'70+'}

In [ ]:
from math import tan
#Setting age ranges
#0 - 5, 5 - 10, 10 - 15, 15 - 20, 20 - 30, 30 - 40, 40 - 50, 50 - 70, 70+
y_age_range = y_age

count_inx_0 = 0
count_inx_1 = 0
count_inx_2 = 0
count_inx_3 = 0
count_inx_4 = 0
count_inx_5 = 0
count_inx_6 = 0
count_inx_7 = 0
count_inx_8 = 0

limit = 1000

for idx, age in np.ndenumerate(y_age_range):
  if age <= 5:
    count_inx_0 += 1
    if count_inx_0 < limit:
      y_age_range[idx] = 0
      count1 += 1
    else:
      y_age_range[idx] = -1

  elif age > 5 and age <= 10:
    count_inx_1 += 1
    if count_inx_1 < limit:
      y_age_range[idx] = 1
    else:
      y_age_range[idx] = -1

  elif age > 10 and age <= 15:
    count_inx_2 += 1
    if count_inx_2 < limit:
      y_age_range[idx] = 2
    else:
      y_age_range[idx] = -1

  elif age > 15 and age <= 20:
    count_inx_3 += 1
    if count_inx_3 < limit:
      y_age_range[idx] = 3
    else:
      y_age_range[idx] = -1

  elif age > 20 and age <= 30:
    count_inx_4 += 1
    if count_inx_4 < limit:
      y_age_range[idx] = 4
      count2 += 1
    else:
      y_age_range[idx] = -1

  elif age > 30 and age <= 40:
    count_inx_5 += 1
    if count_inx_5 < limit:
      y_age_range[idx] = 5
    else:
      y_age_range[idx] = -1

  elif age > 40 and age <= 50:
    count_inx_6 += 1
    if count_inx_6 < limit:
      y_age_range[idx] = 6
    else:
      y_age_range[idx] = -1

  elif age > 50 and age <= 70:
    count_inx_7 += 1
    if count_inx_7 < limit:
      y_age_range[idx] = 7
    else:
      y_age_range[idx] = -1

  elif age > 70:
    count_inx_8 += 1
    if count_inx_8 < limit:
      y_age_range[idx] = 8
      count3 += 1
    else:
      y_age_range[idx] = -1


In [ ]:
# Usuwanie elementów, które nie weszły (index -1)
y_age_range_ = y_age_range
indexes_out = []
a = 0
for idx, num in np.ndenumerate(y_age_range_):
  if num == -1:
    indexes_out.append(idx)
len(indexes_out)

In [ ]:
# Usuwanie indeksów z y
y_age_no_out = np.delete(y_age_range_, indexes_out)

y_age_no_out

In [ ]:
#  Usuwanie tych samych indeksów z x
X_ = X
X_no_out = np.delete(X_, indexes_out, 0)
X_no_out.shape

In [ ]:
# Wykres rozkładu
import seaborn as sns
sns.countplot(x = y_age_no_out)

In [ ]:
# Zwolnienie pamięci zaalokowej dla X
import gc
del(X)
gc.collect()

In [ ]:
# Podział zbioru
Xage_train, Xage_test, yage_train, yage_test = train_test_split(X_no_out, y_age_no_out,
                                                    test_size = 0.2,
                                                    random_state = 4)
yage_train[:10]

In [ ]:
# Zwolnienie pamięci zaalokowej dla X_no_out
import gc
del(X_no_out)
gc.collect()

In [ ]:
#  Operację kodowania kategorycznych danych "one hot encoding"
yage_train_hot = []

for i in yage_train:
  yage_train_hot.append(np.eye(9)[i])

yage_train_hot[:10]

In [ ]:
# Sieć
class ConvolutionalNetwork_age(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,  50, 3, 1)
        self.conv2 = nn.Conv2d(50, 100, 3, 1)
        self.conv3 = nn.Conv2d(100, 150, 3, 1)
        self.conv4 = nn.Conv2d(150, 200, 3, 1)
        self.fc1 = nn.Linear(10*10*200, 3000) #(((((200-2)/2)-2))/2)-2))/2) = 23,.. (4->10)
        self.fc2 = nn.Linear(3000, 300)
        self.fc3 = nn.Linear(300,9) #9-liczba klas wieków

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 10*10*200)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.softmax(X, dim=1)

In [ ]:
model_age = ConvolutionalNetwork_age().to(device)
model_age

In [ ]:
loss_function_age = nn.CrossEntropyLoss()
optimizer_age = torch.optim.Adam(model_age.parameters(), lr=0.001)

## **Trening i testowanie**

In [ ]:
BATCH_SIZE = 80
EPOCHS = 200
MODELPATHAGE = "/content/drive/MyDrive/Projekt_Zespołowy/model_age.pth"

targets = []
preds = []

best_accuracy = 0.0
yage_train_hot = torch.Tensor(yage_train_hot)

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(Xage_train), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        batch_Xage = Xage_train[i:i+BATCH_SIZE].view(-1, 1, 200, 200).to(device)
        batch_yage = yage_train_hot[i:i+BATCH_SIZE].to(device)
        model_age.zero_grad()

        outputs_age = model_age(batch_Xage)
        loss = loss_function_age(outputs_age, batch_yage)
        loss.backward()
        optimizer_age.step()    # Does the update

    acc = get_accuracy(model_age, Xage_test, yage_test)
    print(f"Epoch: {epoch+1}. Loss: {loss}. Accuracy: {acc}")

    if acc > best_accuracy:
      best_accuracy = acc
      model_age.to(device)
      torch.save(model_age.state_dict(), MODELPATHAGE)

    print(best_accuracy)

# **Zastosowanie modelu**

In [ ]:
def predict(model, image):
    with torch.no_grad():
        net_out = model(image.to(device).view(-1, 1, 200, 200))[0]  # returns a list,
        predicted_class = torch.argmax(net_out)
        print('net_out: ', net_out)
    return predicted_class.item()

In [ ]:
model_age = ConvolutionalNetwork_age().cuda()
model_age.load_state_dict(torch.load('/content/drive/MyDrive/Projekt_Zespołowy/model_age.pth'))

In [ ]:
def Example(index):
  predicted_age_range = predict(model_age, X[index])
  plt.imshow(X[index], cmap="Greys_r")
  print(f'Predicted age: {age_dictionary[predicted_age_range]},\
   real age: {age_labels[index]}')

In [ ]:
Example(2211)